# LDA - Linear Discriminant Analysis -  clase multiple
## Baza de date - recunoastere obiecte - ImgTstObjClassif_lab10_11

In [1]:
import os  
import cv2 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append('../_PyUtilFcts/')
import ColorSpaceFunctions as clsFcts

# In[1]  functia pentru formarea vectorului de trasaturi
def ImgFeatExtr(fileName, rszFct, clSpCod,imgComps):
  
    # citire imagine si conversie BGR to RGB
    imgRGB = cv2.cvtColor(cv2.imread(fileName) , cv2.COLOR_BGR2RGB) 
    newShape = (int(imgRGB.shape[0]*rszFct),int(imgRGB.shape[1]*rszFct))
    imgRsz = cv2.resize(imgRGB, newShape, interpolation = cv2.INTER_NEAREST)
    if (NewColorSpace=='R/G/B'):
        ImgClrSp = imgRsz
    elif (NewColorSpace=='gray'):
        ImgClrSp = cv2.cvtColor(imgRsz , cv2.COLOR_RGB2GRAY) 
    else:
        ImgClrSp = clsFcts.ConvertColorSpace(imgRsz, 'R/G/B' , NewColorSpace)
    ImgClrSp = ImgClrSp/255
    if (NewColorSpace=='gray'):
        Img1D = np.reshape(ImgClrSp[:,:],(ImgClrSp.shape[0]*ImgClrSp.shape[1],))
    else:
        Img1D = np.reshape(ImgClrSp[:,:,imgComps],(ImgClrSp.shape[0]*ImgClrSp.shape[1] *imgComps.shape[0],))
    # plt.imshow(imgRsz); plt.show();
    return Img1D

In [3]:
# In[2] Setare parametrii
# Setare director imagini - subdirectoare pe obiecte 
# - 10 exemple pentru fiecare obiect cu ~40 views 
    
pathData = '/Users/galoscar07/Documents/master2k20-2k22/2nd Semester/SIAI/Laborator/_TestImages/ImgTstObjClassif_lab10_11/'

# Folosim clasificatoare binare - clasificam doar 2 obiecte odata
# Numele obiectelor - se pot adauga cate obiecte se doresc 
objName = ['apple','tomato','pear'] 
# objName = ['apple','tomato','pear','cow','horse','dog'] 


# Imaginile obiect se redimensioneaza - miscsoreaza
#   in clasificare folosim varianta redusa
ResizeFactor = 0.15 # valoare intre [0,1] - pentru micsorare imagine

# Setare numar de imagini care vor fi considerate la testare 
# (retul imaginilor vor fi utilizare la antrenare)
test_size = 0.5     # 0.3 / 0.5 / 0.7 

# Setare spatiu de culoare in care este reprezentata imaginea
SelClrSp = 1 # ['R/G/B' = 0, 'Y/Cr/Cb' = 1, 'H/S/V' = 2, 'L/a/b' = 3, 'gray' = 4] 

# Setare componete  pentru spatiu de culoare care se considera in clasificare
imgComps = np.array([0,1,2]) # [C1 = 0, C2 = 1, C3 = 2]

In [4]:
ColorSpacesList = ['R/G/B', 'Y/Cr/Cb', 'H/S/V', 'L/a/b', 'gray'] 
NewColorSpace = ColorSpacesList[SelClrSp] 

print('\n Spatiu de culoare utilizat: '+ NewColorSpace)
print(' Componente utilizate: ', np.array(NewColorSpace.split('/'))[imgComps])
print(' Imaginile vor fi redimensionate/ miscsorate cu factorul de: ', ResizeFactor)
# print(' Setul de date vor fi impartite: ', 100-test_size*100,' % train', test_size*100,' % test' )
print(' Clasele considerate: ', objName)
# In[3] Citire imagini si organizare set de date pentru antrenare/testare
# Fiecare imagine este reprezentata sub forma unui vector de trasaturi 1D, pentru clasificare

# Citim toate imaginile din directorul pathTrn 
# care contin cele 2 obiecte/clase din objName 
# fiecare obiect are mai multe directoare fiecare cu ~40 views/ obiect 
x = []; y=[]; 
for i in range(0,len(objName)): #consideram fiecare forma
    for j in range(1,11): # nr de imagini pentru fiecare forma
        dirName = pathData + objName[i] + str(j)
        dirFiles = os.listdir(dirName)      #all filenames of that particular dir -- image
        for k in range(0,len(dirFiles)):
            fileName = dirName+"/"+dirFiles[k]
            if (os.path.isfile(fileName)):
                imgFeat1D = ImgFeatExtr(fileName, ResizeFactor, NewColorSpace, imgComps)
                x.append(imgFeat1D)
                y.append(i)

# X vectorul de trasaturi 
X = np.asarray(x) 
# y vector cu labelurile fiecarei imagini
y = np.asarray(y) 



 Spatiu de culoare utilizat: Y/Cr/Cb
 Componente utilizate:  ['Y' 'Cr' 'Cb']
 Imaginile vor fi redimensionate/ miscsorate cu factorul de:  0.15
 Clasele considerate:  ['apple', 'tomato', 'pear']


In [5]:
# In[4] Applying LDA - clasificator multiclasa 
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics

# divizare date in set de antrenare si set de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 0)

print(' Setul de date vor fi impartite: ', X_train.shape[0],' train',  X_test.shape[0],' test' )

classifierOvO = OneVsOneClassifier(LDA(n_components = 1))
y_scoreOvO = classifierOvO.fit(X_train, y_train).decision_function(X_test)
y_predOvO = np.argmax(y_scoreOvO, axis=1)

classifierOvR = OneVsRestClassifier(LDA(n_components = 1))
y_scoreOvR = classifierOvR.fit(X_train, y_train).decision_function(X_test)
y_predOvR = np.argmax(y_scoreOvR, axis=1)

print('\n Accuracy LDA OneVsOneClassifier: ', metrics.accuracy_score(y_test, y_predOvO))
print(' Confusion Matrix: \n', metrics.confusion_matrix(y_test,y_predOvO))
print('\n Accuracy LDA OneVsRestClassifier: ', metrics.accuracy_score(y_test, y_predOvR))
print(' Confusion Matrix: \n', metrics.confusion_matrix(y_test,y_predOvR))

 Setul de date vor fi impartite:  615  train 615  test

 Accuracy LDA OneVsOneClassifier:  0.9869918699186991
 Confusion Matrix: 
 [[198   7   0]
 [  1 211   0]
 [  0   0 198]]

 Accuracy LDA OneVsRestClassifier:  0.983739837398374
 Confusion Matrix: 
 [[201   4   0]
 [  6 206   0]
 [  0   0 198]]
